In [2]:
text = """모처럼 전국에 비가 내리고 있습니다.
대부분 밤까지 계속되기 때문에 종일 우산이 필요하겠는데요.
비의 양도 많고 바람도 강하게 불기 때문에 작은 우산 말고 큰 우산으로 챙기는 게 더 좋습니다.
특히 제주와 남해안에서 비바람이 강합니다."""

In [3]:
from nltk.tokenize import sent_tokenize

text = sent_tokenize(text) # 문장 단위로 토큰화
print(text)

['모처럼 전국에 비가 내리고 있습니다.', '대부분 밤까지 계속되기 때문에 종일 우산이 필요하겠는데요.', '비의 양도 많고 바람도 강하게 불기 때문에 작은 우산 말고 큰 우산으로 챙기는 게 더 좋습니다.', '특히 제주와 남해안에서 비바람이 강합니다.']


In [4]:
from konlpy.tag import Okt

okt = Okt()
text2 = []
for txt in text:
    t = okt.nouns(txt) # 한글 형태소 분석. 명사 추출
    text2.append(t)

text2

[['모처럼', '전국', '비'],
 ['대부분', '밤', '계속', '때문', '종일', '우산'],
 ['비', '양도', '바람', '불기', '때문', '우산', '우산', '게', '더'],
 ['제주', '남해안', '비바람']]

In [5]:
from konlpy.tag import Okt

okt = Okt()
text2 = []
for txt in text:
    morph = okt.pos(txt)
    text2.append(morph)

text3 = []
for text in text2:
    line = []
    for word, tag in text:
        if tag in ['Noun','Adjective']: # 명사와 형용사 같이
            line.append(word)
    text3.append(line)

text3

[['모처럼', '전국', '비', '있습니다'],
 ['대부분', '밤', '계속', '때문', '종일', '우산', '필요하겠는데요'],
 ['비',
  '양도',
  '많고',
  '바람',
  '강하게',
  '불기',
  '때문',
  '작은',
  '우산',
  '우산',
  '게',
  '더',
  '좋습니다'],
 ['제주', '남해안', '비바람', '강합니다']]

In [8]:
vocab = {} # 단어 사전
sentences = []
stop_words = ['더', '게']

for txt in text3:
    result = []
    for word in txt:
        if word not in stop_words:
            result.append(word) # 불용어가 아니고,
            if word not in vocab:
                vocab[word] = 0 # 새로운 단어면 추가
            vocab[word] += 1 # 기존에 있던 단어면 카운트 추가
    sentences.append(result)

sentences

[['모처럼', '전국', '비', '있습니다'],
 ['대부분', '밤', '계속', '때문', '종일', '우산', '필요하겠는데요'],
 ['비', '양도', '많고', '바람', '강하게', '불기', '때문', '작은', '우산', '우산', '좋습니다'],
 ['제주', '남해안', '비바람', '강합니다']]

In [9]:
# 빈도수

print(vocab)
print(vocab["우산"])

{'모처럼': 1, '전국': 1, '비': 2, '있습니다': 1, '대부분': 1, '밤': 1, '계속': 1, '때문': 2, '종일': 1, '우산': 3, '필요하겠는데요': 1, '양도': 1, '많고': 1, '바람': 1, '강하게': 1, '불기': 1, '작은': 1, '좋습니다': 1, '제주': 1, '남해안': 1, '비바람': 1, '강합니다': 1}
3


In [10]:
word_to_index = {}
i = 0
for word in vocab:
    if vocab[word] > 1:
        i = i + 1
        word_to_index[word] = i

word_to_index

{'비': 1, '때문': 2, '우산': 3}

In [11]:
# 단어 집합에 없는 단어는 word_to_index에 OOV라는 단어를 추가해서 OOV로 처리
# 기타 단어 OOV의 인덱스 4로 설정

word_to_index['OOV'] = len(word_to_index) + 1

encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)

encoded

# 이 방법이 바로 정수 인코딩
# 단어 하나하나에 번호를 지정하는 것
# 단점은 가중치가 붙을 수 있다는 것

[[4, 4, 1, 4],
 [4, 4, 4, 2, 4, 3, 4],
 [1, 4, 4, 4, 4, 4, 2, 4, 3, 3, 4],
 [4, 4, 4, 4]]

In [12]:
import numpy as np

words = np.hstack(sentences) # 2차원 데이터를 1차원으로
words

array(['모처럼', '전국', '비', '있습니다', '대부분', '밤', '계속', '때문', '종일', '우산',
       '필요하겠는데요', '비', '양도', '많고', '바람', '강하게', '불기', '때문', '작은', '우산',
       '우산', '좋습니다', '제주', '남해안', '비바람', '강합니다'], dtype='<U7')

In [13]:
from collections import Counter

vocab = Counter(words) # 단어의 출현빈도를 쉽게 계산하는 클래스
print(vocab)
print(vocab["우산"])

Counter({'우산': 3, '비': 2, '때문': 2, '모처럼': 1, '전국': 1, '있습니다': 1, '대부분': 1, '밤': 1, '계속': 1, '종일': 1, '필요하겠는데요': 1, '양도': 1, '많고': 1, '바람': 1, '강하게': 1, '불기': 1, '작은': 1, '좋습니다': 1, '제주': 1, '남해안': 1, '비바람': 1, '강합니다': 1})
3


In [14]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('우산', 3), ('비', 2), ('때문', 2), ('모처럼', 1), ('전국', 1)]

In [15]:
word_to_index = {}
i = 0
for (word, frequency) in vocab:
    i = i + 1
    word_to_index[word] = i
word_to_index

{'우산': 1, '비': 2, '때문': 3, '모처럼': 4, '전국': 5}

In [ ]:
# 원핫인코딩

In [16]:
from konlpy.tag import Okt

okt = Okt()
token = okt.morphs("나는 학교에 간다 나는 집에 간다")
print(token)

['나', '는', '학교', '에', '간다', '나', '는', '집', '에', '간다']


In [17]:
word2index = {}
for idx, voca in enumerate(token):
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '학교': 2, '에': 3, '간다': 4, '집': 5}


In [18]:
def one_hot_encoding(word, word2index):
    one_hot_vector = [0]*(len(word2index)) # 전체 단어의 개수만큼 0으로 채운 리스트
    index = word2index[word] # 해당하는 단어의 인덱스 찾기
    one_hot_vector[index] = 1 # 해당하는 단어만 1, 나머지는 0
    return one_hot_vector

one_hot_encoding("학교", word2index)

[0, 0, 1, 0, 0, 0]

In [19]:
# 추가된 코드

key_list = word2index.keys()
print(key_list)
for key in key_list:
    print(key, one_hot_encoding(key, word2index))

dict_keys(['나', '는', '학교', '에', '간다', '집'])
나 [1, 0, 0, 0, 0, 0]
는 [0, 1, 0, 0, 0, 0]
학교 [0, 0, 1, 0, 0, 0]
에 [0, 0, 0, 1, 0, 0]
간다 [0, 0, 0, 0, 1, 0]
집 [0, 0, 0, 0, 0, 1]


In [ ]:
# Bag of Words

In [20]:
from konlpy.tag import Okt
import re

okt = Okt()
token = re.sub("[.!#~]","","비가 오니 마음이 차분해지네요. 요즘 너무 더웠어요. 비가 오니 마음이 기쁘네요.")
token = okt.morphs(token)

word2index = {}
bow = []
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
        bow.insert(len(word2index)-1,1)
    else:
        index = word2index.get(voca)
        bow[index] = bow[index]+1
    
print(word2index)
print(bow)

{'비': 0, '가': 1, '오니': 2, '마음': 3, '이': 4, '차분해지네요': 5, '요즘': 6, '너무': 7, '더웠어요': 8, '기쁘네요': 9}
[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['모처럼 전국에 비가 내리고 있습니다.']
line = ['전국에 비가']
vector = CountVectorizer()
vector.fit(corpus)
print(vector.vocabulary_)
print(vector.transform(line).toarray())

{'모처럼': 1, '전국에': 4, '비가': 2, '내리고': 0, '있습니다': 3}
[[0 0 1 0 1]]


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["""모처럼 전국에 비가 내리고 있습니다.
대부분 밤까지 계속되기 때문에 종일 우산이 필요하겠는데요.
비의 양도 많고 바람도 강하게 불기 때문에 작은 우산 말고 큰 우산으로 챙기는 게 더 좋습니다.
특히 제주와 남해안에서 비바람이 강합니다."""]

vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
{'모처럼': 9, '전국에': 22, '비가': 13, '내리고': 4, '있습니다': 20, '대부분': 5, '밤까지': 11, '계속되기': 2, '때문에': 6, '종일': 24, '우산이': 19, '필요하겠는데요': 28, '비의': 15, '양도': 16, '많고': 7, '바람도': 10, '강하게': 0, '불기': 12, '작은': 21, '우산': 17, '말고': 8, '우산으로': 18, '챙기는': 26, '좋습니다': 25, '특히': 27, '제주와': 23, '남해안에서': 3, '비바람이': 14, '강합니다': 1}
